# Libraries

In [1]:
# IO
import os
import csv
import pathlib
from pathlib import Path
import chardet
import warnings

# Utilities
import numpy as np 
import pandas as pd
import copy

# Preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from torchtext.vocab import Vocab
from collections import Counter
from torch.nn.utils.rnn import pad_sequence

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Modelling and training
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
## Metrics
from sklearn.metrics import confusion_matrix
warnings.filterwarnings("ignore", category=FutureWarning)

/home/iridhexx/miniconda3/envs/project2/lib/python3.12/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/home/iridhexx/miniconda3/envs/project2/lib/python3.12/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


Download `nltk` resources (only needs to run once per machine)

In [2]:
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/iridhexx/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/iridhexx/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Classes

Early stopping class (pytorch does not have a built in early stopping callback)

In [ ]:
class EarlyStopping:
    def __init__(self, monitor='val_loss', patience=1, restore_best_weights=True, mode='min'):
        self.monitor = monitor
        self.patience = patience
        self.restore_best_weights = restore_best_weights
        self.mode = mode  # 'min' for loss, 'max' for accuracy, etc.
        self.best_score = None
        self.counter = 0
        self.early_stop = False
        self.best_model_state = None

    def __call__(self, current_score, model):
        # Determine if the current score is better
        if self.best_score is None:
            self.best_score = current_score
            if self.restore_best_weights:
                self.best_model_state = copy.deepcopy(model.state_dict())
        elif (self.mode == 'min' and current_score < self.best_score) or \
             (self.mode == 'max' and current_score > self.best_score):
            self.best_score = current_score
            self.counter = 0
            if self.restore_best_weights:
                self.best_model_state = copy.deepcopy(model.state_dict())
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

    def restore(self, model):
        if self.restore_best_weights and self.best_model_state:
            model.load_state_dict(self.best_model_state)

## Models

In [ ]:
class LoSiento(nn.Module):
    def __init__(self, max_features, embed_dim=128, lstm_out=196, output_classes=3):
        super(LoSiento, self).__init__()
        
        self.embedding = nn.Embedding(num_embeddings=max_features, embedding_dim=embed_dim)
        self.spatial_dropout = nn.Dropout2d(p=0.5)  # Approximate SpatialDropout1D
        self.lstm = nn.LSTM(embed_dim, lstm_out, batch_first=True, dropout=0.3)
        
        self.dropout_1 = nn.Dropout(p=0.2)
        self.dense_1 = nn.Linear(lstm_out, 100)
        self.dropout_2 = nn.Dropout(p=0.4)
        self.output_layer = nn.Linear(100, output_classes)

    def forward(self, x):
        x = self.embedding(x)                      # (batch, seq_len, embed_dim)
        x = x.permute(0, 2, 1)                     # For Dropout2d: (batch, embed_dim, seq_len)
        x = self.spatial_dropout(x)
        x = x.permute(0, 2, 1)                     # Back to (batch, seq_len, embed_dim)
        
        x, _ = self.lstm(x)                        # LSTM returns (output, (h_n, c_n))
        x = x[:, -1, :]                            # Get the output from the last timestep
        
        x = self.dropout_1(x)
        x = F.relu(self.dense_1(x))
        x = self.dropout_2(x)
        x = self.output_layer(x)
        return F.softmax(x, dim=1)         

# Functions

The encoding of the text file is automatically detected here with some confidence. It could be extracted from terminal using
```
file -I file.txt
```

Terminal method (not os agnostic):

In [8]:
#!file -i ~/Documents/StudyResources/IML/Project/Part2/_data/FinancialPhraseBank-v1.0/Sentences_50Agree.txt

Python method:

In [6]:
def extract_sentences_from_file(filepath):
    """Given an input txt file, extract sentences 
    and associated sentiments

    Args:
        filepath (string): path to string 

    Returns:
        list: list of sentences and sentiments
    """
    sentences = []
    # automatically detect endoding (best guess)
    with open(filepath, 'rb') as file:
        encoding = chardet.detect(file.read())['encoding']
    # read and split
    with open(filepath, 'r', encoding=encoding) as file:
        for line in file:
            line = line.strip()
            if '.@' in line:
                sentence, sentiment = line.rsplit('.@', 1)
                sentence = sentence.strip()
                sentiment = sentiment.strip().lower()
                sentence = fix_common_mojibake(sentence)
                sentences.append((sentence, sentiment))
    return sentences

In [7]:
def txt_to_csv(txt_path, output_csv):
    """Takes a file path as input and generates a .csv file
    which contains sentences and sentiments as columns, extracted
    from the .txt file corresponding to the path

    Args:
        txt_path (string): path to the .txt file
        output_csv (string): path to desired output .csv file
    """
    # Skip processing if CSV already exists
    if os.path.exists(output_csv):
        print(f"{output_csv} already exists. Skipping processing.")
        return

    sentences = extract_sentences_from_file(txt_path)

    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Sentence', 'Sentiment'])  # Header
        writer.writerows(sentences)
    print(f"Processed files and wrote output to {output_csv}")


This function is needed as the text files seem to be corrupted, for example one sentence is:
```
Clothing retail chain Sepp+ñl+ñ 's sales increased by 8 % to EUR 155.2 mn , and operating profit rose to EUR 31.1 mn from EUR 17.1 mn in 2004
```
The characters `+ñ` are a result of choosing the wrong encoding; in fact, here the detected encoding is latin-1, and those characters correspond to `ä` in utf-8. The function below is used to fix these common mistakes

In [9]:
def fix_common_mojibake(text):
    """Function to manually handle encoding errors

    Args:
        text (string): input sentence

    Returns:
        string: output corrected sentence
    """
    replacements = {
        '+ñ': 'ä',
        '+í': 'é',
        '+ô': 'ö',
        '+ü': 'ü'
        }
    for wrong, right in replacements.items():
        text = text.replace(wrong, right)
    return text

This function summarizes the common preprocessing pipeline for natural language models.

In [ ]:
def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())

    # Remove stop words
    filtered_tokens = [token for token in tokens if token not in stopwords.words('english')]

    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    # Join the tokens back into a string
    processed_text = ' '.join(lemmatized_tokens)

    return processed_text